In [4]:
!ls ~/workspace/turb

ls: cannot access '~/workspace/turb/data02_02/jhd-001-nc-zarr': No such file or directory


In [2]:
%cd ~/workspace/turb/data01_01

/home/idies/workspace/Storage/ariel4/persistent


In [ ]:
# creatuple list of folder on filedb to which data should be copied
# you may want to change this to avoid certain volumes
folders=[f'/home/idies/workspace/turb/data{str(f).zfill(2)}_{str(d).zfill(2)}/zarr/ncar-zarr/'  for f in range(1,13) for d in range(1,4)]
folders[:5]

In [ ]:
# Avoiding 7-2 and 9-2 - they're too full as of May 2023
# folders.remove("/home/idies/workspace/turb/data02_02/zarr/ncar-zarr/")
folders.remove("/home/idies/workspace/turb/data09_02/zarr/ncar-zarr/")
folders.remove("/home/idies/workspace/turb/data07_02/zarr/ncar-zarr/") # This is already created

In [ ]:
# # Chatgpt - create folders above - skip if already created

# import os

# def create_folders(paths):
#     for path in paths:
#         os.makedirs(path, exist_ok=True)
#         print(f"Folder created at {path}")

# create_folders(folders)


 # Create folders


In [20]:
# a GlobusAuthorizer is an auxiliary object we use to wrap the token. In
# more advanced scenarios, other types of GlobusAuthorizers give us
# expressive power
authorizer = globus_sdk.AccessTokenAuthorizer(TRANSFER_TOKEN)
tc = globus_sdk.TransferClient(authorizer=authorizer)

In [40]:
# defining endpoints. Note, these may have to be activated explicitly, can do that through the UI.
source_endpoint='jhuidies#dmztest'   # change to your source endpoint at flatiron
source=tc.get_endpoint(source_endpoint)

In [28]:
# f
r=tc.endpoint_autoactivate(endpoint_id=target_endpoint)
while (r["code"] == "AutoActivationFailed"):
    print("Endpoint requires manual activation, please open "
      "the following URL in a browser to activate the "
      "endpoint:")
    print("https://www.globus.org/app/endpoints/%s/activate"% target_endpoint)
    input("Press ENTER after activating the endpoint:")
    r = tc.endpoint_autoactivate(target_endpoint, if_expires_in=3600)

Endpoint requires manual activation, please open the following URL in a browser to activate the endpoint:
https://www.globus.org/app/endpoints/jhuidies#dmztest/activate


Press ENTER after activating the endpoint: 


In [3]:
# create list of folder on filedb to which data should be copied
# you may want to change this to avoid certain volumes
folders=[f'/sciserver/filedb{str(f).zfill(2)}-{str(d).zfill(2)}/turb/zarr/'  for f in range(1,13) for d in range(1,4)]
folders[:5]

['/sciserver/filedb01-01/turb/zarr/',
 '/sciserver/filedb01-02/turb/zarr/',
 '/sciserver/filedb01-03/turb/zarr/',
 '/sciserver/filedb02-01/turb/zarr/',
 '/sciserver/filedb02-02/turb/zarr/']

In [4]:
# Avoiding 7-2 and 9-2
folders.remove("/sciserver/filedb07-02/turb/zarr/")
folders.remove("/sciserver/filedb09-02/turb/zarr/")

In [7]:
len(folders)

34

# Write to folders

In [17]:
# Edited from ChatGPT
import os
import shutil

source_directory = '/home/idies/workspace/turb/data02_02/ariel-6-timestep-ncar-zarr'
destination_directory = '/home/idies/workspace/turb/'
num_disks = 36

# Create destination directories on each disk
for i in range(num_disks):
    disk_directory = os.path.join(destination_directory, f'disk{i+1}')
    os.makedirs(disk_directory, exist_ok=True)

# Traverse the source directory and distribute files to destination directories
for root, dirs, files in os.walk(source_directory):
    for file in files:
        source_path = os.path.join(root, file)

        # Determine the destination disk
        disk_index = sum([ord(c) for c in file]) % num_disks
        destination_disk = os.path.join(destination_directory, f'disk{disk_index + 1}')

        # Create the corresponding directory structure on the destination disk
        destination_path = os.path.join(destination_disk, os.path.relpath(root, source_directory))
        os.makedirs(destination_path, exist_ok=True)

        # Copy the file to the destination disk
        shutil.copy(source_path, destination_path)


task_id = e77304ea-2d09-11ec-95dc-853490a236f9


# TODO Fix Zarr Metadata

In [10]:
z = zarr.open("/home/idies/workspace/turb/data02_03/zarr/ncar-zarr")
z.info

Name,/
Type,zarr.hierarchy.Group
Read-only,False
Store type,zarr.storage.DirectoryStore
No. members,0
No. arrays,0
No. groups,0


In [11]:
zarr.consolidate_metadata("/home/idies/workspace/turb/data02_03/zarr/ncar-zarr/e")

PathNotFoundError: nothing found at path ''